In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../')
from utils import *

pd.set_option('display.max_columns', None)

In [2]:
DATA_PATH = '../../data/'
PASSES_DF_PATH = DATA_PATH + 'passes_df.pkl'
WYSCOUT_TO_SKILLCORNER = DATA_PATH + 'wyscout2skillcorner.csv'

In [3]:
passes_df = pd.read_pickle(PASSES_DF_PATH)
wyscout2skillcorner = pd.read_csv(WYSCOUT_TO_SKILLCORNER).drop(columns='id')

In [4]:
passes_df.rename(columns={'player.id': 'player.id.wyscout', 'pass.recipient.id': 'pass.recipient.id.wyscout', 'tracking.player_id': 'tracking.player.id.skillcorner'}, inplace=True)
passes_df.drop(columns=['team.id', 'opponentTeam.id'], inplace=True)

In [5]:
# An id value of 0 doesn't map to any player. Look into it.
passes_df = passes_df[passes_df['pass.recipient.id.wyscout'] != 0]

In [6]:
passes_df = passes_df.merge(wyscout2skillcorner[['player_id_wy', 'player_id_sk']],
                            left_on='player.id.wyscout', 
                            right_on='player_id_wy', 
                            how='left')

passes_df = passes_df.merge(wyscout2skillcorner[['player_id_wy', 'player_id_sk']],
                            left_on='pass.recipient.id.wyscout', 
                            right_on='player_id_wy', 
                            how='left')

In [7]:
passes_df.rename(columns={'player_id_sk_x': 'player.id.skillcorner', 'player_id_sk_y': 'pass.recipient.id.skillcorner'}, inplace=True)
passes_df.drop(columns=['player_id_wy_x', 'player_id_wy_y'] , inplace=True)

In [10]:
passes_df['tracking.is_teammate'] = (passes_df['team.name'] == passes_df['tracking.team_name'])
passes_df['tracking.is_self'] = (passes_df['player.id.skillcorner'] == passes_df['tracking.player.id.skillcorner'])

Validate coordinate transformation. Wyscout's [location.x, location.y] are standardized accoridng to pitch dimensions. Skillcorner's [tracking.x, tracking.y] are absolute measurements. They should match.

In [12]:
passes_df[passes_df['tracking.is_self']][['location.x.wyscout', 'location.y.wyscout', 'location.x', 'location.y', 'tracking.x', 'tracking.y']]

,location.x.wyscout,location.y.wyscout,location.x,location.y,tracking.x,tracking.y
21,51,50,-1.058824,0.000000,-0.70,0.90
39,42,83,8.470588,21.616842,9.87,11.72
56,29,60,22.235294,7.046154,22.60,7.11
79,25,28,26.470588,-14.660000,22.84,-18.80
100,32,50,19.058824,0.000000,20.52,-3.75
...,...,...,...,...,...,...
19543,46,65,4.235294,10.382222,-2.42,-12.15
19566,40,89,10.588235,25.987368,-6.34,-28.24
19587,61,91,-11.647059,27.444211,4.19,-28.35
19606,72,30,-23.294118,-13.437778,20.55,11.67


In [14]:
passes_df['responsibility'] = passes_df.apply(responsibility, axis=1, pass_length_factor=1)
passes_df['responsibility'] = np.where(passes_df['tracking.is_teammate'], 0, passes_df['responsibility'])

In [ ]:
# skillcorner_cols = [col for col in passes_df.columns if col.startswith('tracking.')]
# wyscout_cols = [col for col in passes_df.columns if col not in skillcorner_cols]
# passes_df = passes_df.groupby(wyscout_cols).agg(list)